<img src="https://i.ibb.co/K5n24Nd/1.png"  alt="KCode Banner"  />

<a href="https://www.linkedin.com/in/kauebr/"><img src="https://cdn-icons-png.flaticon.com/512/174/174857.png" width="30" ></a> <a href="mailto: kauemandarino@gmail.com"><img src="https://cdn-icons-png.flaticon.com/512/5968/5968534.png" width="30" ></a><a href="
https://api.whatsapp.com/send?phone=5541991835800&text=Oi"> <img src="https://imagepng.org/wp-content/uploads/2017/08/whatsapp-icone-1.png" width="30" ></a><a href="https://github.com/kauebr/"><img src="https://github.githubassets.com/images/modules/logos_page/Octocat.png" width="30" ></a>



# Tratamento de dados, valores das ações x Selic

## Objetivo:
* Analisar o valor da cotação da ação de empresas, relacionando com o valor da taxa Selic no mesmo período.

## Método:
* Definir o período 2000 ~ 2022(22 anos) OK
* Minerar a cotação Selic que vem sempre no último dia útil de cada mês
* Minerar os valores das cotações das ações empresas: `GGBR4.SA Gerdau preferenciais`, `GGBR3.SA Gerdau Metalúrgica preferenciais`,
                                                      `VALE3.SA Vale`, `USIM5.SA Usiminas` e `CSNA3.SA CSN` OK
* Reduzir as cotações e ficar apenas com as referentes ao último dia útil de cada mês para podermos comparar com o valor da Selic;
* Plotar gráficos para análise;



In [1]:
import pandas as pd
from pandas_datareader import data as web
from datetime import date

In [2]:
# Função que puxa as cotações, detecta, por variação discreta, qual é o último dia de cada mês e retorna apenas os valores das ações referentes ao último dia útil de cada mês.

def GLC(cod_empresa, data_inicial, data_final): # Gerar lista de cotações (mm-dd-aaaa)
# Puxando a cotação mãe(todo o período)
    cotacao = web.DataReader(cod_empresa, data_source='yahoo', start=data_inicial, end=data_final)['Adj Close']
    cotacao = cotacao.to_frame()
    cotacao.reset_index(inplace=True)
    cotacao.rename(columns={'Date':'Data'}, inplace=True)
    # Adcionando colunas âncora "meses" e "ano"
    cotacao = cotacao.assign(Meses = 0)
    cotacao['Meses'] = cotacao['Data'].dt.month
    cotacao['Ano'] = cotacao['Data'].dt.year
    # Calculando o último dia(útil) de cada mês, baseado na variação discreta entre o valor das linhas dos mesês(quando muda o mês, ela salva o index em um relatório)
    meses = cotacao['Meses']
    c = 1
    lista_dias = []
    for loop in range(0, len(meses)-1):
        atual = meses[c]
        anterior = meses[c-1]
        if atual != anterior:
            lista_dias.append(c-1)
        c += 1
    # Utilizando os valores do index, para selecionar apenas os dias desejados
    cotacao_final = pd.DataFrame()
    lista_datas = list()
    lista_cotacoes = list()
    lista_referencias = list()
    for i in lista_dias: 
        lista_datas.append(cotacao.iloc[i]['Data'])
        lista_cotacoes.append(cotacao.iloc[i]['Adj Close'])
        ano = cotacao.iloc[i]['Ano']
        mes = cotacao.iloc[i]['Meses']
        lista_referencias.append(str(f'{mes}-{ano}'))
        # Criando o nosso dataframe para abrigar o resultado
    cotacao_final['Data'] = lista_datas
    cotacao_final['Cotacoes'] = lista_cotacoes
    cotacao_final['Ref'] = lista_referencias
    return cotacao_final


In [3]:
empresas = {'GGBR4.SA': 'Gerdau preferenciais', 'GGBR3.SA': 'Gerdau Metalúrgica preferenciais',
             'VALE3.SA': 'Vale', 'USIM5.SA': 'Usiminas',  'CSNA3.SA': 'CSN'}

In [4]:
cotacoes_acoes = dict()
inicio = '01-01-2000'
fim =  '08-31-2022'

cotacoes_acoes['Gerdau Pref'] = GLC('GGBR4.SA', inicio, fim)
cotacoes_acoes['Gerdau Met Pre'] = GLC('GGBR3.SA', inicio, fim)
cotacoes_acoes['Vale'] = GLC('VALE3.SA', inicio, fim)
cotacoes_acoes['Usiminas'] = GLC('USIM5.SA', inicio, fim)
cotacoes_acoes['CSN'] = GLC('CSNA3.SA', inicio, fim)

In [5]:
cotacoes_acoes

{'Gerdau Pref':           Data    Cotacoes     Ref
 0   2000-01-31  268.362549  1-2000
 1   2000-02-29  263.639282  2-2000
 2   2000-03-31  244.210022  3-2000
 3   2000-04-28  234.012115  4-2000
 4   2000-05-31    1.252358  5-2000
 ..         ...         ...     ...
 266 2022-03-31   29.354248  3-2022
 267 2022-04-29   26.648436  4-2022
 268 2022-05-31   28.365076  5-2022
 269 2022-06-30   21.733269  6-2022
 270 2022-07-29   23.785044  7-2022
 
 [271 rows x 3 columns],
 'Gerdau Met Pre':           Data   Cotacoes     Ref
 0   2000-01-31   0.219444  1-2000
 1   2000-02-29   0.231984  2-2000
 2   2000-03-31   0.200635  3-2000
 3   2000-04-28   0.219444  4-2000
 4   2000-05-31   0.980719  5-2000
 ..         ...        ...     ...
 266 2022-03-31  22.675571  3-2022
 267 2022-04-29  20.724682  4-2022
 268 2022-05-31  22.144821  5-2022
 269 2022-06-30  17.251686  6-2022
 270 2022-07-29  18.953648  7-2022
 
 [271 rows x 3 columns],
 'Vale':           Data   Cotacoes     Ref
 0   2000-01-31   

Uma vez os dados minerados, organizados e filtrados, é só partir para análise.